## Python 对象的三个基本属性

在 Python 中，**一切皆对象**。每个对象都有三个基本属性：

1. **Identity（身份）**：对象的唯一标识，在对象生命周期内不变。  
   - 在 CPython 中，`id()` 返回的是对象的内存地址。
2. **Type（类型）**：对象属于什么类型（如 `int`、`list`、`str` 等）。
3. **Value（值）**：对象实际存储的数据内容。

> 📌 对象 ≠ 内存地址 + 数据，而是一个**整体实体**，它**拥有**身份、类型和值。

In [7]:
# 创建一个列表对象
a = [1, 2, 3]

# 查看对象的三个属性
print("对象 a 的身份（id）:", id(a))
print("对象 a 的类型（type）:", type(a))
print("对象 a 的值（value）:", a)

对象 a 的身份（id）: 1502325129984
对象 a 的类型（type）: <class 'list'>
对象 a 的值（value）: [1, 2, 3]


## 引用 vs 新对象

- `b = a`：`b` 和 `a` 指向**同一个对象** → `id(b) == id(a)`
- `c = a.copy()`：创建**新对象** → `id(c) != id(a)`，但值相同

In [8]:
b = a               # 引用
c = a.copy()        # 浅拷贝（新对象）

print("a 和 b 是同一对象？", id(a) == id(b), "（等价于 b is a）")
print("a 和 c 是同一对象？", id(a) == id(c))

# 验证 is 运算符
print("b is a ?", b is a)
print("c is a ?", c is a)

a 和 b 是同一对象？ True （等价于 b is a）
a 和 c 是同一对象？ False
b is a ? True
c is a ? False


## 扩展：NumPy 的 `view()`（浅视图）

- `view()` 创建一个**新 ndarray 对象**（不同 id）
- 但**共享底层数据内存**
- 修改视图会影响原数组

In [12]:
import numpy as np

a_np = np.array([10, 20, 30])
c_np = a_np.view()

print("NumPy: a_np 和 c_np 是同一对象？", a_np is c_np)
print("它们的 id 相同吗？", id(a_np) == id(c_np))
print("它们共享数据吗？", np.shares_memory(a_np, c_np))

# 修改视图
c_np[0] = 999
print("修改 c_np 后，a_np 变为:", a_np)
print("修改后c_np变为：", c_np)

NumPy: a_np 和 c_np 是同一对象？ False
它们的 id 相同吗？ False
它们共享数据吗？ True
修改 c_np 后，a_np 变为: [999  20  30]
修改后c_np变为： [999  20  30]


# 引用、浅拷贝 与 深拷贝

我们已经知道：

- **引用（`b = a`）**：两个变量指向同一个对象。
- **浅拷贝（`a.copy()` 或 `copy.copy(a)`）**：创建新对象，但**嵌套对象仍共享引用**。

但当对象包含**嵌套可变对象**（如列表中的列表）时，浅拷贝可能不够用。

这时就需要 **深拷贝（deep copy）**！

> ✅ **深拷贝**：递归地复制对象及其所有嵌套对象，**完全独立**。

In [2]:
import copy

# 创建一个包含嵌套列表的对象
original = [[1, 2], [3, 4]]

# 三种方式
reference = original                 # 引用
shallow = copy.copy(original)        # 浅拷贝
deep = copy.deepcopy(original)       # 深拷贝

print("原始对象 ID:", id(original))
print("引用 ID     :", id(reference))
print("浅拷贝 ID   :", id(shallow))
print("深拷贝 ID   :", id(deep))
print()
print("外层对象是否相同？")
print("reference is original :", reference is original)
print("shallow   is original :", shallow is original)
print("deep      is original :", deep is original)

原始对象 ID: 1579543991488
引用 ID     : 1579543991488
浅拷贝 ID   : 1579543994752
深拷贝 ID   : 1579543994176

外层对象是否相同？
reference is original : True
shallow   is original : False
deep      is original : False


In [3]:
# 修改浅拷贝中的嵌套列表
# original = [[1, 2], [3, 4]]
shallow[0][0] = 'X'

print("修改 shallow[0][0] = 'X' 后：")
print("original =", original)
print("shallow  =", shallow)
print("deep     =", deep)
print()

# 再修改深拷贝，看是否影响原对象
deep[1][1] = 'Y'
print("再修改 deep[1][1] = 'Y' 后：")
print("original =", original)
print("deep     =", deep)

修改 shallow[0][0] = 'X' 后：
original = [['X', 2], [3, 4]]
shallow  = [['X', 2], [3, 4]]
deep     = [[1, 2], [3, 4]]

再修改 deep[1][1] = 'Y' 后：
original = [['X', 2], [3, 4]]
deep     = [[1, 2], [3, 'Y']]


## 三种方式对比总结

| 方式        | 是否新对象？ | 嵌套对象是否独立？ | 适用场景 |
|-------------|-------------|------------------|--------|
| `b = a`（引用） | ❌ 否        | ❌ 共享全部        | 快速别名 |
| `copy.copy(a)`（浅拷贝） | ✅ 是（外层） | ❌ 嵌套仍共享      | 扁平结构或只读嵌套 |
| `copy.deepcopy(a)`（深拷贝） | ✅ 是（递归） | ✅ 完全独立        | 嵌套可变对象，需完全隔离 |

> ⚠️ 深拷贝性能开销较大，仅在必要时使用。

In [4]:
# 查看嵌套子对象的 ID
# 
print("original[0] ID :", id(original[0]))
print("shallow[0] ID  :", id(shallow[0]))
print("deep[0] ID     :", id(deep[0]))
print()
print("浅拷贝的嵌套对象是否共享？", shallow[0] is original[0])   # True
print("深拷贝的嵌套对象是否独立？", deep[0] is original[0])      # False

original[0] ID : 1579543993536
shallow[0] ID  : 1579543993536
deep[0] ID     : 1579543992768

浅拷贝的嵌套对象是否共享？ True
深拷贝的嵌套对象是否独立？ False


## 🧠 引用、浅拷贝 与 深拷贝：内存地址对比总结

| 操作                          | 外层对象地址是否改变？ | 嵌套对象地址是否改变？ | 说明                     |
|-----------------------------|----------------------|----------------------|------------------------|
| `b = a`（引用）               | ❌ 不变                | ❌ 不变                | 两个变量指向**同一个对象**       |
| `copy.copy(a)`（浅拷贝）      | ✅ 改变                | ❌ 不变（共享）         | 创建**新外层对象**，但嵌套对象仍**共享引用** |
| `copy.deepcopy(a)`（深拷贝）  | ✅ 改变                | ✅ 改变（完全独立）     | **递归复制**，所有层级都创建新对象   |

> 💡 **关键理解**：  
> **浅拷贝 → 外层是新对象（地址变），内层仍是原对象（地址不变）**  
> 因此，修改嵌套的可变元素（如子列表）会影响原始对象！

> 🎯 **记住**：  
> **浅拷贝 ≠ 完全独立**！只有外层“壳”是新的，里面的“内容”还是原来的引用。

## 最佳实践建议

- 如果对象是**不可变的**（如 `(1, 2, [3, 4])` 中的元组，但注意内部有可变项），要小心！
- 对于**纯简单类型**（如 `[1, 2, 3]`），浅拷贝和深拷贝效果相同。
- 对于**包含可变嵌套结构**（如列表、字典、自定义对象），若需完全隔离，**必须用 `deepcopy`**。
- 避免对大型对象频繁深拷贝，可能影响性能。

> 💡 小技巧：不确定时，用 `id()` 或 `is` 检查是否共享！